In [ ]:
\list

In [ ]:
SELECT keywords_all FROM public_job_listings.data_nerd_jobs LIMIT 100;

In [ ]:
SELECT
  job_id,
  JSON_EXTRACT_SCALAR(keyword, '$.element') AS keyword
FROM
  public_job_listings.data_nerd_jobs,
  UNNEST(JSON_EXTRACT_ARRAY(keywords_all, '$.list')) AS keyword
LIMIT 100;

In [ ]:
SELECT
  job_id,
  REGEXP_EXTRACT_ALL(TO_JSON_STRING(keywords_all), r'"element": "([^"]+)"') AS keywords
FROM
  public_job_listings.data_nerd_jobs
LIMIT 100;

In [ ]:
SELECT
  job_id,
  REGEXP_EXTRACT_ALL(TO_JSON_STRING(keywords_all), r'"element": "([^"]+)"') AS keywords
FROM
  public_job_listings.data_nerd_jobs
LIMIT 100;

In [ ]:
SELECT
  job_id,
  TO_JSON_STRING(keywords_all) AS keywords_json
FROM
  public_job_listings.data_nerd_jobs
LIMIT 100;

In [ ]:
SELECT
  job_id,
  REGEXP_EXTRACT_ALL(TO_JSON_STRING(keywords_all), r'"element": "([^"]+)"') AS keywords
FROM
  public_job_listings.data_nerd_jobs
LIMIT 100;

In [ ]:
import pandas as pd
import json

def extract_keywords(row):
    try:
        keywords = json.loads(row['keywords_all'])['list']
        return [kw['element'] for kw in keywords]
    except:
        return []

# Load the data into a DataFrame
df = sql_df_deoy

# Apply the function to extract keywords
df['keywords'] = df['keywords_all'].apply(extract_keywords)
df[['keywords']].head()

In [ ]:
SELECT JSON_EXTRACT_SCALAR(keywords_all, '$.list[0].element') AS first_keyword FROM public_job_listings.data_nerd_jobs LIMIT 100;

In [ ]:
SELECT
  JSON_EXTRACT(keywords_all, '$.list') AS keywords
FROM public_job_listings.data_nerd_jobs
LIMIT 100;

In [ ]:
import pandas as pd
import json

# Query to fetch the keywords_all column
query = """
SELECT keywords_all
FROM public_job_listings.data_nerd_jobs
LIMIT 100
"""

# Execute the query and load into a DataFrame
keywords_df = sql_df_deoy

# Define a function to extract keywords
def extract_keywords(json_str):
    try:
        keywords_list = json.loads(json_str)['list']
        return [kw['element'] for kw in keywords_list]
    except (TypeError, json.JSONDecodeError):
        return []

# Apply the function to extract keywords
keywords_df['extracted_keywords'] = keywords_df['keywords_all'].apply(extract_keywords)
keywords_df

In [ ]:
SELECT 
    JSON_EXTRACT_SCALAR(keyword, '$.element') AS keyword
FROM 
    public_job_listings.data_nerd_jobs,
    UNNEST(JSON_EXTRACT_ARRAY(keywords_all, '$.list')) AS keyword
LIMIT 100;

In [ ]:
SELECT keyword.element
FROM public_job_listings.data_nerd_jobs,
     UNNEST(JSON_EXTRACT_ARRAY(keywords_all, '$.list')) AS keyword
LIMIT 100;

In [ ]:
SELECT keyword.element
FROM public_job_listings.data_nerd_jobs,
UNNEST(JSON_EXTRACT_ARRAY(keywords_all, '$.list')) AS keyword
LIMIT 100;

In [ ]:
SELECT JSON_QUERY(keywords_all, '$.list') AS keywords_json
FROM public_job_listings.data_nerd_jobs
LIMIT 100;